In [4]:
import langchain_ollama
print(dir(langchain_ollama))


['ChatOllama', 'OllamaEmbeddings', 'OllamaLLM', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'chat_models', 'embeddings', 'llms']


In [1]:
import os

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [16]:
#carregando modelo llm 
llm = OllamaLLM(model="llama3.2:latest", request_timeout=5000.0, temperature=0.0)

In [17]:
#carregango o embedding
embeds_model = OllamaEmbeddings(model="llama3.2:latest")

In [18]:
#carregando o pdf
loader = PyPDFLoader("/home/luan/rag/Resumo_S.O.pdf", extract_images=False)
pages = loader.load_and_split()

In [19]:
#carregando os chunks
text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True
)

chunks = text_spliter.split_documents(pages)

In [21]:
try:
    vector_store = Chroma.from_documents(
        documents=chunks,
        embedding=embeds_model,
        persist_directory="./test2"  # Diretório de persistência
    )
    print("Banco de dados criado e persistido automaticamente.")
except Exception as e:
    print(f"Erro ao criar o banco: {e}")


Banco de dados criado e persistido automaticamente.


In [22]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=retriever
)


In [23]:
def ask(question):
    try:
        # Recuperar documentos relevantes usando o retriever
        context = retriever.invoke(question)
        
        # Passar os documentos e a pergunta para o chain
        response = qa_chain.invoke({"input_documents": context, "query": question})
        
        # Verificando a estrutura completa da resposta
        print("Resposta Completa:", response)  # Imprimindo o retorno para entender sua estrutura
        
        # Ajuste na extração da resposta
        answer = response.get('output_text', "Resposta não encontrada.")
        
        return answer, context  # Retornando a resposta e o contexto
        
    except Exception as e:
        return f"Erro ao processar a pergunta: {e}", None  # Retorna erro e contexto como None


In [25]:

# Interação com o usuário
user_question = input("User: ")
answer, context = ask(user_question)

print("\nAnswer:", answer)
if context:
    print("\nContexto utilizado:")
    for doc in context:
        print(doc)

Resposta Completa: {'input_documents': [Document(metadata={'page': 2, 'source': '/home/luan/rag/Resumo_S.O.pdf', 'start_index': 0}, page_content='ResumoSlide4:\nProcessos:\n● Sãoinstânciasdeprogramasemexecução,contendoinformaçõescomocontadordeprograma,registradoresevariáveis.● Osistemaoperacionalorganizaprogramasemprocessosque,atravésdochaveamento,dãoaimpressãodeexecuçãoparalela.\nThreads:\n● São"miniprocessos"dentrodeummesmoprocesso,compartilhandorecursos,maspermitindoexecuçõesindependentes.● Úteisparaaplicaçõesquerequeremmúltiplasatividadessimultâneas,comoservidoreswebeeditoresdetexto.\nComunicaçãoInterprocesso(IPC):\n● Podeocorrerviamemóriacompartilhada(rápida,massuscetívelaproblemasdesincronização)outrocademensagens(maissegura,porémmaislentaemsistemasdistribuídos).\nProblemasClássicos:\n● Condiçõesdecorrida:Quandoprocessosacessamsimultaneamenteamesmamemória,causandoinconsistências.● Deadlocks:Quandoprocessosficamesperandoindefinidamenteporrecursosunsdosoutros.\nSoluçõesdeSincroniza

In [27]:
print(context[0])

page_content='ResumoSlide4:
Processos:
● Sãoinstânciasdeprogramasemexecução,contendoinformaçõescomocontadordeprograma,registradoresevariáveis.● Osistemaoperacionalorganizaprogramasemprocessosque,atravésdochaveamento,dãoaimpressãodeexecuçãoparalela.
Threads:
● São"miniprocessos"dentrodeummesmoprocesso,compartilhandorecursos,maspermitindoexecuçõesindependentes.● Úteisparaaplicaçõesquerequeremmúltiplasatividadessimultâneas,comoservidoreswebeeditoresdetexto.
ComunicaçãoInterprocesso(IPC):
● Podeocorrerviamemóriacompartilhada(rápida,massuscetívelaproblemasdesincronização)outrocademensagens(maissegura,porémmaislentaemsistemasdistribuídos).
ProblemasClássicos:
● Condiçõesdecorrida:Quandoprocessosacessamsimultaneamenteamesmamemória,causandoinconsistências.● Deadlocks:Quandoprocessosficamesperandoindefinidamenteporrecursosunsdosoutros.
SoluçõesdeSincronização:
● Semáforos:Paragerenciaroacessoarecursoscompartilhados.● Monitores:Estruturasdealtonívelqueevitamproblemascomsemáforos.● Barreiras:Gara